<a href="https://colab.research.google.com/github/jakubmis1998/NeuralNetworks/blob/main/zeroRecognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [389]:
# 0 recognition

import keras.preprocessing.image as kpi
from matplotlib import pyplot as plt
import numpy as np

class Perceptron():
  def __init__(self, learning_rate, iterations, sample_dim, samples_dim):
    """
      Initialization of perceptron.

      Args:
        learning_rate (float) - const learning rate.
        iterations (integer) - learning iterations amount.
        sample_dim ((integer, integer)) - single sample dimensions.
        samples_dim ((integer, integer)) - samples dimensions.
    """
    self.learning_rate = learning_rate
    self.iterations = iterations
    self.sample_dim = sample_dim
    self.samples_dim = samples_dim
    # Random THETA [-1, 1)
    self.THETA = np.random.default_rng().uniform(-1, 1, 1)[0]
    # Random weights [-1, 1)
    self.weights = np.random.default_rng().uniform(-1, 1, self.sample_dim)
  
  def train(self, training_data, expected_values):
    """
      Perceptron learning based on training and expected data.

      Args:
        train_data (NumPy 2D array) - training data set.
        expected_values (NumPy 2D array) - set of correct(expected) values.
    """
    global_life_time = 0
    local_life_time = 0
    classified = np.full((self.samples_dim[0], self.samples_dim[1]), False)
    best_weights = self.weights

    for index in range(self.iterations):
      # Coordinates of random digit
      i = np.random.randint(self.sample_dim[0])  # Row
      j = np.random.randint(self.sample_dim[1])  # Column

      # Predict
      ERR = self.predict(training_data[i][j], expected_values[i][j])

      # Unclassified sample
      if ERR != 0:
        # Reset local life time
        local_life_time = 0
        # Reset classified samples
        classified = np.full((self.samples_dim[0], self.samples_dim[1]), False)
        # Weights and THETA calculate
        self.weights += learning_rate * ERR * training_data[i][j]
        self.THETA -= learning_rate * ERR
      # Classified sample
      else:
        # Increase life time
        local_life_time += 1
        # Classify sample
        classified[i][j] = True
        # Store current winner and set new better weights
        if global_life_time < local_life_time:
          global_life_time = local_life_time
          best_weights = self.weights
    
    # Set the best weights for perceptron
    self.weights = best_weights

  def predict(self, digit, expected_value):
    """
      Checks if given data returns expected value based on the learned perceptron.

      Args:
        digit (NumPy array): Digit.
        expected_value (number): 0(No) / 1(Yes).
    """
    O = 1 if np.dot(self.weights.flatten(), digit.flatten()) - self.THETA >= 0 else -1
    ERR = expected_value - O
    return ERR
  
  def calculate_efficiency(self, all_digits):
    """
      Returns efficiency of perceptron in %.

      Args:
        all_digits (NumPy array): All of the digits.
    """
    samples_number = 1.0
    correctly_classified = 0.0
    # Calculate all samples number
    for dim in self.samples_dim:
      samples_number *= dim

    for i in range(self.samples_dim[0]):
      for j in range(self.samples_dim[1]):
        # 0 Recognition
        if i == 0:
          if perceptron.predict(all_digits[i][j], 1) == 0:
            correctly_classified += 1
        else:
          if perceptron.predict(all_digits[i][j], -1) == 0:
            correctly_classified += 1

    return "Perceptron efficiency: {}%".format(correctly_classified / samples_number * 100)

  def split_into_2D(self, image_name):
    # 2D array with single pictures
    array_2D = np.array([])

    # Load the main image
    main_img = kpi.load_img(image_name)

    # Convert to numpy array
    all_images = kpi.img_to_array(main_img)  # RGB 0-255 values
    all_images /= 255  # RGB 0-1 values

    # Split entire array with all digits to single images
    for i in range(self.samples_dim[0]):
      row = np.array([])
      for j in range(self.samples_dim[1]):
        # Create single 7x5 and remove last RGB canal
        single_image = all_images[
          i*self.sample_dim[0]:(i+1)*self.sample_dim[0],
          j*self.sample_dim[1]:(j+1)*self.sample_dim[1]
        ][:, :, 0]
        # Append single 7x5 picture
        row = np.append(row, single_image)
      # Append 8 pictures (row of 7x5 single pictures)
      array_2D = np.append(array_2D, row)
    
    # Make 2D array with 7x5 pictures
    array_2D = array_2D.reshape(
      self.samples_dim[0],
      self.samples_dim[1],
      self.sample_dim[0],
      self.sample_dim[1]
    )
    print("Single image shape:", array_2D[0][0].shape)
    print("Row shape:", array_2D[0].shape)
    print("Array shape:", array_2D.shape)
    print("Samples number:", array_2D.size // (self.sample_dim[0] * self.sample_dim[1]))

    return array_2D

def show_image(img):
  """Show 2D image from NumPy array"""
  plt.imshow(img)

if __name__ == "__main__":
  # Perceptron initialization
  perceptron = Perceptron(
    learning_rate=0.1,
    iterations=5000,
    sample_dim=(7, 5),
    samples_dim=(10, 8)
  )

  # An 2D array of single pictures
  images = perceptron.split_into_2D('mnist.png')

  # 8 samples 2D of digits == 0
  A = images[0]
  # 72 samples 2D of digits != 0
  B = images[1:10]

  # Samples for perceptron learning
  E = np.zeros((
    perceptron.samples_dim[0],
    perceptron.samples_dim[1],
    perceptron.sample_dim[0],
    perceptron.sample_dim[1])
  )  # 80 samples
  E[0] = A  # 8 good samples
  E[1:10] = B  # 72 bad samples

  # True results for perceptron learning
  T = np.zeros((perceptron.samples_dim[0], perceptron.samples_dim[1]))
  T[0] = 1
  T[1:10] = -1

  # Train perceptron with data and expected values
  perceptron.train(E, T)

  # Calculate perceptron efficienct running it on all samples
  print(perceptron.calculate_efficiency(E))



Single image shape: (7, 5)
Row shape: (8, 7, 5)
Array shape: (10, 8, 7, 5)
Samples number: 80
Perceptron efficiency: 95.0%
